In [1]:
import tensorflow as tf
# from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K

from src import data
from src import error_function

jobs = 6 # number of cores
config = tf.ConfigProto(intra_op_parallelism_threads=jobs,
                     inter_op_parallelism_threads=jobs,
                     allow_soft_placement=True,
                     device_count={'CPU': jobs})
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [9]:
## loading the model
model = keras.models.load_model('data/models/CRNN.h5',         
        custom_objects={ 
            'class_mae': error_function.class_mae,
            'exp': keras.backend.exp
        })

/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/asr/lib/python3.7/site-packages/keras/engine/base_layer.py:1179: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="conv1", activity_regularizer=None, trainable=True, activation="relu", kernel_size=(3, 3), filters=64, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot_uniform", kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_bias=True)`
  return cls(**config)
/Users/jordynaus/Documents/Artificial Intelligence/Master/2021 Sem. 2/Automatic Speech Recognition/asr/lib/python3.7/site-packages/keras/engine/base_layer.py:1179: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(name="conv2", activity_regularizer=None, trainable=True, activation="relu", kernel_size=(3, 3), filters=32, strides=[1, 1], padding="valid", data_format="channels_first", kernel_initializer="glorot

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero1 (ZeroPadding2D)        (None, 1, 500, 201)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 498, 199)      640       
_________________________________________________________________
conv2 (Conv2D)               (None, 32, 496, 197)      18464     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 32, 165, 65)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 128, 163, 63)      36992     
_________________________________________________________________
conv4 (Conv2D)               (None, 64, 161, 61)       73792     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 64, 53, 20)       

In [4]:
## load dataset
train_dataset, val_dataset = data.create_tensorflow_dataset('./data/trainset100/stft/', batch_size=32, val_split = 0.3)

Found 7860 images belonging to 11 classes.
Found 3363 images belonging to 11 classes.


In [5]:
train_dataset[0][0]

array([[[[0.01176471, 0.01960784, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         [0.01176471, 0.01960784, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         [0.02352941, 0.01960784, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         ...,
         [0.01176471, 0.01568628, 0.01568628, ..., 0.03137255,
          0.02352941, 0.03137255],
         [0.01568628, 0.01176471, 0.01568628, ..., 0.02745098,
          0.04313726, 0.04705883],
         [0.01568628, 0.01176471, 0.01568628, ..., 0.02745098,
          0.04313726, 0.04705883]]],


       [[[0.01176471, 0.01176471, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         [0.01176471, 0.01176471, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         [0.01176471, 0.01176471, 0.01176471, ..., 0.01176471,
          0.01176471, 0.01176471],
         ...,
         [0.0627451 , 0.0627451 , 0.05882353, ..., 0.04705883,
          0.03921569, 0

In [6]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs = 2)

Epoch 1/2
246/246 [==============================] - 1963s 8s/step - loss: 1.9864 - accuracy: 0.3324 - class_mae: 1.7724 - val_loss: 1.4749 - val_accuracy: 0.4737 - val_class_mae: 0.7925
Epoch 2/2
246/246 [==============================] - 1948s 8s/step - loss: 1.1877 - accuracy: 0.4818 - class_mae: 0.8618 - val_loss: 1.2519 - val_accuracy: 0.5019 - val_class_mae: 0.7642


In [8]:
model.save('data/models/CRNN_100.h5', overwrite=True, include_optimizer=True)